In [1]:
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DataConversionWarning)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.linear_model import SGDRegressor, LinearRegression, SGDClassifier
from xgboost.sklearn import XGBClassifier, XGBRegressor, XGBRFRegressor, XGBRFClassifier
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 30)

In [2]:
df = pd.read_csv('ver1.csv', index_col=['id'])
# display(df.columns)
# df.drop(columns=['production_countries', 'production_companies', 'genres', 'release_date', 
#                  'popularity_norm', 'year', 'original_language', 'fr', 'ru', 'es', 'hi', 
#                  'ja', 'it', 'ko', 'cn', 'zh', 'others', 'month', 'date'], inplace=True)

df.drop(columns=['production_countries', 'production_companies', 'genres', 'release_date', 
                 'popularity_norm', 'others'], inplace=True)

df_norm = df

# df_norm_mean = df_norm.mean()
# df_norm_std = df_norm.std()

# df_norm = (df_norm - df_norm_mean) / df_norm_std
# df_norm['en'] = df['en']
display(df_norm)


X = df_norm.drop(columns=['revenue'])
y = df_norm[['revenue']]
y_bin, bins = pd.qcut(y['revenue'], 10,retbins=True, labels=[*range(10)])

X_train, X_test = train_test_split(df_norm, test_size = 0.1, random_state=123)

X_train = pd.DataFrame(data=X_train)
y_train = X_train[['revenue']]
X_train = X_train.drop(columns=['revenue'])
X_test = pd.DataFrame(data=X_test)
y_test = X_test[['revenue']]
X_test = X_test.drop(columns=['revenue'])
y_train_bin = pd.cut(y_train['revenue'], bins=bins, labels=[*range(10)], include_lowest=True)
y_test_bin = pd.cut(y_test['revenue'], bins=bins, labels=[*range(10)], include_lowest=True)

,budget,original_language,popularity,revenue,runtime,vote_average,vote_count,year,month,date,quarter,en,fr,ru,es,hi,ja,it,ko,cn,zh
id,,,,,,,,,,,,,,,,,,,,,
5,4000000,1,22.876230,4300000,98.0,6.5,530,1995,12,9,4,1,0,0,0,0,0,0,0,0,0
11,11000000,1,126.393695,775398007,121.0,8.1,6624,1977,5,25,2,1,0,0,0,0,0,0,0,0,0
12,94000000,1,85.688789,940335536,100.0,7.6,6122,2003,5,30,2,1,0,0,0,0,0,0,0,0,0
13,55000000,1,138.133331,677945399,142.0,8.2,7927,1994,7,6,3,1,0,0,0,0,0,0,0,0,0
14,15000000,1,80.878605,356296601,122.0,7.9,3313,1999,9,15,3,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365222,36000000,9,19.167377,156844753,105.0,6.5,379,2015,12,19,4,0,0,0,0,0,0,0,0,1,0
375290,4500000,5,2.551559,32000000,126.0,7.3,57,2016,1,22,1,0,0,0,0,1,0,0,0,0,0
376659,20000000,1,42.512296,183936074,100.0,6.3,1252,2016,7,28,3,1,0,0,0,0,0,0,0,0,0


In [3]:
ada_reg_dt = AdaBoostRegressor()

In [4]:
ada_reg_dt_dict = cross_validate(ada_reg_dt ,X, y, cv=20, return_train_score=True, return_estimator=True)

In [5]:
display(ada_reg_dt_dict['train_score'])
display(ada_reg_dt_dict['test_score'])

array([0.54573346, 0.62021398, 0.60453308, 0.66895124, 0.50959107,
       0.56945542, 0.33043339, 0.39815531, 0.47303218, 0.38723415,
       0.63582648, 0.49479127, 0.46848404, 0.52478206, 0.46798513,
       0.30571894, 0.40086645, 0.45307633, 0.48367821, 0.6180879 ])

array([ 0.49371948,  0.58870308,  0.48326582,  0.68339289, -0.4991628 ,
       -0.51044904, -0.0977828 , -0.56332238,  0.0658755 , -0.88407019,
        0.12087429, -0.14894721,  0.45951775,  0.39908917,  0.42892687,
        0.37050635,  0.09828594,  0.63286529,  0.30074666,  0.51119697])

In [6]:
# ada_reg_sgd = AdaBoostRegressor(base_estimator=SGDRegressor(learning_rate='adaptive', max_iter=3000))

In [7]:
# ada_reg_sgd_dict = cross_validate(ada_reg_sgd ,X, y, cv=20, return_train_score=True, return_estimator=True)

In [8]:
# display(ada_reg_sgd_dict['train_score'])
# display(ada_reg_sgd_dict['test_score'])

array([-1.04095087e+25, -5.19100684e+25, -2.43938007e+25, -5.63163798e+24,
       -2.31137627e+24, -1.54497960e+24, -7.56471665e+25, -2.74321528e+24,
       -1.87910740e+26, -1.51824087e+24, -1.18715717e+25, -3.92540129e+25,
       -3.83092155e+25, -3.73516677e+25, -2.57112183e+23, -5.82387760e+25,
       -1.28035539e+25, -4.19391963e+24, -7.97640909e+24, -3.03218061e+23])

array([-6.39889235e+24, -2.96229672e+25, -2.53692300e+25, -8.80520835e+24,
       -7.24814578e+24, -4.69015081e+24, -1.12144095e+26, -6.41802036e+24,
       -3.44066020e+26, -3.25774928e+24, -2.17562737e+25, -6.87997735e+25,
       -1.52761038e+25, -2.37034999e+25, -4.08658292e+23, -7.35191381e+25,
       -2.16446650e+25, -2.71824047e+24, -6.47916690e+24, -2.93944666e+23])

In [9]:
ada_reg_lr = AdaBoostRegressor(base_estimator=LinearRegression())

In [10]:
ada_reg_lr_dict = cross_validate(ada_reg_lr ,X, y, cv=20, return_train_score=True, return_estimator=True)

In [11]:
display(ada_reg_lr_dict['train_score'])
display(ada_reg_lr_dict['test_score'])

array([0.65412527, 0.65104619, 0.65424028, 0.65960305, 0.62606606,
       0.63522805, 0.64224528, 0.62341899, 0.62229238, 0.66828962,
       0.62328327, 0.64409536, 0.6640268 , 0.62177286, 0.61921973,
       0.64146924, 0.66443681, 0.60702331, 0.6593535 , 0.62161209])

array([0.56467409, 0.58720121, 0.59739098, 0.62949888, 0.31629902,
       0.33740284, 0.56696054, 0.47423246, 0.57602284, 0.56050601,
       0.44720989, 0.52029631, 0.67376816, 0.65073045, 0.60557141,
       0.60285974, 0.17764786, 0.81698235, 0.46750393, 0.63928689])

In [12]:
ada_clf_dt = AdaBoostClassifier()

In [13]:
ada_clf_dt_dict = cross_validate(ada_clf_dt ,X, y_bin, cv=20, return_train_score=True, return_estimator=True)

In [14]:
display(ada_clf_dt_dict['train_score'])
display(ada_clf_dt_dict['test_score'])

array([0.29877036, 0.30109671, 0.30608175, 0.31339315, 0.30309073,
       0.3000997 , 0.28780326, 0.29910269, 0.30109671, 0.29444998,
       0.29478232, 0.32070455, 0.30674643, 0.30142905, 0.30674643,
       0.28647391, 0.30265781, 0.29744949, 0.30573037, 0.29976814])

array([0.24375   , 0.2125    , 0.24375   , 0.3       , 0.2875    ,
       0.2875    , 0.34375   , 0.3875    , 0.3125    , 0.3125    ,
       0.31875   , 0.26875   , 0.275     , 0.3125    , 0.35      ,
       0.2625    , 0.3081761 , 0.24666667, 0.30666667, 0.22      ])

In [15]:
y_train_bin, X_train

(id
 9779      4
 598       3
 1588      6
 1125      7
 4547      8
          ..
 27029     1
 290595    7
 13183     8
 9428      5
 10160     2
 Name: revenue, Length: 2852, dtype: category
 Categories (10, int64): [0 < 1 < 2 < 3 ... 6 < 7 < 8 < 9],
            budget  original_language  popularity  runtime  vote_average  \
 id                                                                        
 9779     25000000                  1   17.173720    119.0           6.3   
 598       3300000                  0   44.356711    130.0           8.1   
 1588     28000000                  1   14.754281     98.0           5.8   
 1125     70000000                  1   14.486311    134.0           6.6   
 4547     48000000                  1   53.043124    111.0           6.5   
 ...           ...                ...         ...      ...           ...   
 27029     3500000                  1    3.281509    149.0           7.4   
 290595  115000000                  1   60.467984    114.0     

In [16]:
xgb_clf = XGBClassifier(verbosity=1, n_jobs=14, gpu_id=0, eval_metrics='merror')

In [17]:
xgb_clf_gscv = GridSearchCV(xgb_clf, param_grid={
    'max_dept': [3, 8],
    'booster': ['gbtree'],
    'objective': ['multi:softmax', 'multi:softprob'],
    'n_estimators': [600]
}, return_train_score=True, cv=20, verbose=3)
xgb_clf_gscv.fit(X, y_bin)

Fitting 20 folds for each of 4 candidates, totalling 80 fits
[CV] booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax, score=(train=0.894, test=0.225), total=   2.2s
[CV] booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.3s remaining:    0.0s


[CV]  booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax, score=(train=0.898, test=0.219), total=   2.0s
[CV] booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.4s remaining:    0.0s


[CV]  booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax, score=(train=0.905, test=0.225), total=   2.0s
[CV] booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax 
[CV]  booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax, score=(train=0.903, test=0.306), total=   2.1s
[CV] booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax 
[CV]  booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax, score=(train=0.902, test=0.294), total=   2.1s
[CV] booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax 
[CV]  booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax, score=(train=0.902, test=0.287), total=   2.1s
[CV] booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax 
[CV]  booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax, score=(train=0.899, test=0.331), total=   2.1s
[CV] booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax 


[CV]  booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax, score=(train=0.903, test=0.306), total=   2.1s
[CV] booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax 
[CV]  booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax, score=(train=0.902, test=0.294), total=   2.0s
[CV] booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax 
[CV]  booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax, score=(train=0.902, test=0.287), total=   2.0s
[CV] booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax 
[CV]  booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax, score=(train=0.899, test=0.331), total=   2.1s
[CV] booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax 
[CV]  booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax, score=(train=0.903, test=0.381), total=   2.1s
[CV] booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax 


[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  2.9min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=20, error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, eval_metrics='merror',
                                     gamma=0, gpu_id=0, learning_rate=0.1,
                                     max_delta_step=0, max_depth=3,
                                     min_child_weight=1, missing=None,
                                     n_estimators=100, n_jobs=14, nthread=None,
                                     objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='warn', n_jobs=None,
             param_grid={'booster': ['gbtree'], 'max_dept': [3, 8],
                         'n

In [ ]:
pd.DataFrame(xgb_clf_gscv.cv_results_)[['mean_test_score']]

In [19]:
xgb_rf_clf = XGBRFClassifier(verbosity=1, n_jobs=14, gpu_id=0, eval_metrics='merror')

In [20]:
xgb_rf_clf_gscv = GridSearchCV(xgb_rf_clf, param_grid={
    'max_dept': [3, 8],
    'booster': ['gbtree'],
    'objective': ['multi:softmax', 'multi:softprob'],
    'n_estimators': [600]
}, return_train_score=True, cv=20, verbose=3)
xgb_rf_clf_gscv.fit(X, y_bin)

Fitting 20 folds for each of 4 candidates, totalling 80 fits
[CV] booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax, score=(train=0.345, test=0.263), total=   1.9s
[CV] booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.9s remaining:    0.0s


[CV]  booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax, score=(train=0.344, test=0.275), total=   1.9s
[CV] booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.9s remaining:    0.0s


[CV]  booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax, score=(train=0.346, test=0.244), total=   1.9s
[CV] booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax 
[CV]  booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax, score=(train=0.350, test=0.319), total=   1.9s
[CV] booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax 
[CV]  booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax, score=(train=0.340, test=0.306), total=   1.9s
[CV] booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax 
[CV]  booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax, score=(train=0.354, test=0.312), total=   1.9s
[CV] booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax 
[CV]  booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax, score=(train=0.348, test=0.362), total=   1.9s
[CV] booster=gbtree, max_dept=3, n_estimators=600, objective=multi:softmax 


[CV]  booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax, score=(train=0.350, test=0.319), total=   2.0s
[CV] booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax 
[CV]  booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax, score=(train=0.340, test=0.306), total=   2.0s
[CV] booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax 
[CV]  booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax, score=(train=0.354, test=0.312), total=   2.0s
[CV] booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax 
[CV]  booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax, score=(train=0.348, test=0.362), total=   2.0s
[CV] booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax 
[CV]  booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax, score=(train=0.346, test=0.356), total=   2.0s
[CV] booster=gbtree, max_dept=8, n_estimators=600, objective=multi:softmax 


[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:  2.6min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=20, error_score='raise-deprecating',
             estimator=XGBRFClassifier(base_score=0.5, colsample_bylevel=1,
                                       colsample_bynode=0.8, colsample_bytree=1,
                                       eval_metrics='merror', gamma=0, gpu_id=0,
                                       learning_rate=1, max_delta_step=0,
                                       max_depth=3, min_child_weight=1,
                                       missing=None, n_estimators=100,
                                       n_jobs=14, nthread=None,
                                       objective='binary:logistic',
                                       random_state=0, reg_alpha=0,
                                       reg_lambda=1, scale_pos_weight=1,
                                       seed=None, silent=None, subsample=0.8,
                                       verbosity=1),
             iid='warn', n_jobs=None,
             param_grid={'booster': ['gbtree'], 'm

In [21]:
pd.DataFrame(xgb_rf_clf_gscv.cv_results_)[['mean_test_score']]

,mean_test_score
0,0.294099
1,0.294099
2,0.294099
3,0.294099


In [22]:
xgb_reg = XGBRegressor(eval_metrics='rmse', verbosity=1, n_jobs=14, gpu_id=0)

In [24]:
xgb_reg_gscv = GridSearchCV(xgb_reg, param_grid={
#     'max_dept': [*range(3, 9)],
    'feature_selector': ['cyclic', 'greedy', 'thrifty'],
    'max_depth': [*range(3, 9, 2)],
    'top_k': [*range(1, 5)],
    'objective': ['reg:squarederror']
    
}, return_train_score=True, cv=20, verbose=3)
xgb_reg_gscv.fit(X, y)

Fitting 20 folds for each of 36 candidates, totalling 720 fits
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1, score=(train=0.854, test=0.620), total=   0.1s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1, score=(train=0.851, test=0.674), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1, score=(train=0.857, test=0.669), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1, score=(train=0.855, test=0.708), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1, score=(train=0.850, test=0.527), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1, score=(train=0.851, test=0.605), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1, score=(train=0.852, test=0.604), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1, score=(train=0.849, test=0.695), total=   0.0s
[CV] feature_selector=cyclic, max_depth=

[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3, score=(train=0.855, test=0.708), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3, score=(train=0.850, test=0.527), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3, score=(train=0.851, test=0.605), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3, score=(train=0.852, test=0.604), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3, score=(train=0.849, test=0.695), total=   0.0s
[CV] feature_selector=cyclic, max_depth=

[CV]  feature_selector=cyclic, max_depth=5, objective=reg:squarederror, top_k=1, score=(train=0.931, test=0.719), total=   0.1s
[CV] feature_selector=cyclic, max_depth=5, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=5, objective=reg:squarederror, top_k=1, score=(train=0.928, test=0.585), total=   0.1s
[CV] feature_selector=cyclic, max_depth=5, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=5, objective=reg:squarederror, top_k=1, score=(train=0.931, test=0.529), total=   0.1s
[CV] feature_selector=cyclic, max_depth=5, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=5, objective=reg:squarederror, top_k=1, score=(train=0.930, test=0.635), total=   0.1s
[CV] feature_selector=cyclic, max_depth=5, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=5, objective=reg:squarederror, top_k=1, score=(train=0.928, test=0.700), total=   0.1s
[CV] feature_selector=cyclic, max_depth=

...

[CV]  feature_selector=thrifty, max_depth=5, objective=reg:squarederror, top_k=4, score=(train=0.930, test=0.655), total=   0.1s
[CV] feature_selector=thrifty, max_depth=5, objective=reg:squarederror, top_k=4 
[CV]  feature_selector=thrifty, max_depth=5, objective=reg:squarederror, top_k=4, score=(train=0.931, test=0.719), total=   0.1s
[CV] feature_selector=thrifty, max_depth=5, objective=reg:squarederror, top_k=4 
[CV]  feature_selector=thrifty, max_depth=5, objective=reg:squarederror, top_k=4, score=(train=0.928, test=0.585), total=   0.1s
[CV] feature_selector=thrifty, max_depth=5, objective=reg:squarederror, top_k=4 
[CV]  feature_selector=thrifty, max_depth=5, objective=reg:squarederror, top_k=4, score=(train=0.931, test=0.529), total=   0.1s
[CV] feature_selector=thrifty, max_depth=5, objective=reg:squarederror, top_k=4 
[CV]  feature_selector=thrifty, max_depth=5, objective=reg:squarederror, top_k=4, score=(train=0.930, test=0.635), total=   0.1s
[CV] feature_selector=thrifty, 

[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=2, score=(train=0.974, test=0.659), total=   0.1s
[CV] feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=2 
[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=2, score=(train=0.974, test=0.679), total=   0.1s
[CV] feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=2 
[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=2, score=(train=0.974, test=0.582), total=   0.1s
[CV] feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=2 
[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=2, score=(train=0.976, test=0.533), total=   0.1s
[CV] feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=2 
[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=2, score=(train=0.975, test=0.616), total=   0.1s
[CV] feature_selector=thrifty, 

[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=4, score=(train=0.974, test=0.659), total=   0.1s
[CV] feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=4 
[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=4, score=(train=0.974, test=0.679), total=   0.1s
[CV] feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=4 
[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=4, score=(train=0.974, test=0.582), total=   0.1s
[CV] feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=4 
[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=4, score=(train=0.976, test=0.533), total=   0.1s
[CV] feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=4 
[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=4, score=(train=0.975, test=0.616), total=   0.1s
[CV] feature_selector=thrifty, 

[Parallel(n_jobs=1)]: Done 720 out of 720 | elapsed:   54.4s finished


GridSearchCV(cv=20, error_score='raise-deprecating',
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, eval_metrics='rmse',
                                    gamma=0, gpu_id=0, importance_type='gain',
                                    learning_rate=0.1, max_delta_step=0,
                                    max_depth=3, min_child_weight=1,
                                    missing=None, n_estimators=100, n_jobs=14,
                                    nthread=None, objecti...near',
                                    random_state=0, reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1),
             iid='warn', n_jobs=None,
             param_grid={'feature_selector': ['cyclic', 'greedy', 'thrifty'],
                  

In [25]:
pd.DataFrame(xgb_reg_gscv.cv_results_)[['mean_test_score']]

,mean_test_score
0,0.679574
1,0.679574
2,0.679574
3,0.679574
4,0.671465
5,0.671465
6,0.671465
7,0.671465
8,0.657778
9,0.657778


In [26]:
xgb_rf_reg = XGBRFRegressor(eval_metrics='rmse', verbosity=1, n_jobs=14, gpu_id=0)

In [27]:
xgb_rf_reg_gscv = GridSearchCV(xgb_rf_reg, param_grid={
    'feature_selector': ['cyclic', 'greedy', 'thrifty'],
    'max_depth': [*range(3, 9, 2)],
    'top_k': [*range(1, 5)],
    'objective': ['reg:squarederror']
    
}, return_train_score=True, cv=20, verbose=3)
xgb_rf_reg_gscv.fit(X, y)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Fitting 20 folds for each of 36 candidates, totalling 720 fits
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1, score=(train=0.711, test=0.621), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1, score=(train=0.709, test=0.606), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1, score=(train=0.717, test=0.566), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1, score=(train=0.704, test=0.713), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s


[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1, score=(train=0.709, test=0.444), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1, score=(train=0.708, test=0.555), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1, score=(train=0.709, test=0.522), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1, score=(train=0.707, test=0.653), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=1, score=(train=0.704, test=0.720), total=   0.0s
[CV] feature_selector=cyclic, max_depth=

[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3, score=(train=0.706, test=0.668), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3, score=(train=0.707, test=0.614), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3, score=(train=0.704, test=0.796), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3, score=(train=0.714, test=0.503), total=   0.0s
[CV] feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3 
[CV]  feature_selector=cyclic, max_depth=3, objective=reg:squarederror, top_k=3, score=(train=0.700, test=0.697), total=   0.0s
[CV] feature_selector=cyclic, max_depth=

...

[CV]  feature_selector=thrifty, max_depth=5, objective=reg:squarederror, top_k=4, score=(train=0.789, test=0.577), total=   0.1s
[CV] feature_selector=thrifty, max_depth=5, objective=reg:squarederror, top_k=4 
[CV]  feature_selector=thrifty, max_depth=5, objective=reg:squarederror, top_k=4, score=(train=0.775, test=0.784), total=   0.1s
[CV] feature_selector=thrifty, max_depth=5, objective=reg:squarederror, top_k=4 
[CV]  feature_selector=thrifty, max_depth=5, objective=reg:squarederror, top_k=4, score=(train=0.786, test=0.634), total=   0.1s
[CV] feature_selector=thrifty, max_depth=5, objective=reg:squarederror, top_k=4 
[CV]  feature_selector=thrifty, max_depth=5, objective=reg:squarederror, top_k=4, score=(train=0.785, test=0.696), total=   0.1s
[CV] feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=1 
[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=1, score=(train=0.828, test=0.634), total=   0.1s
[CV] feature_selector=thrifty, 

[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=2, score=(train=0.827, test=0.598), total=   0.1s
[CV] feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=2 
[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=2, score=(train=0.820, test=0.790), total=   0.1s
[CV] feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=2 
[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=2, score=(train=0.826, test=0.643), total=   0.1s
[CV] feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=2 
[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=2, score=(train=0.827, test=0.690), total=   0.1s
[CV] feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=3 
[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=3, score=(train=0.828, test=0.634), total=   0.1s
[CV] feature_selector=thrifty, 

[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=4, score=(train=0.827, test=0.598), total=   0.1s
[CV] feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=4 
[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=4, score=(train=0.820, test=0.790), total=   0.1s
[CV] feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=4 
[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=4, score=(train=0.826, test=0.643), total=   0.1s
[CV] feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=4 
[CV]  feature_selector=thrifty, max_depth=7, objective=reg:squarederror, top_k=4, score=(train=0.827, test=0.690), total=   0.1s


[Parallel(n_jobs=1)]: Done 720 out of 720 | elapsed:   47.6s finished


GridSearchCV(cv=20, error_score='raise-deprecating',
             estimator=XGBRFRegressor(base_score=0.5, colsample_bylevel=1,
                                      colsample_bynode=0.8, colsample_bytree=1,
                                      eval_metrics='rmse', gamma=0, gpu_id=0,
                                      learning_rate=1, max_delta_step=0,
                                      max_depth=3, min_child_weight=1,
                                      missing=None, n_estimators=100, n_jobs=14,
                                      nthread=None, objective='reg:linear',
                                      random_state=0, reg_alpha=0, reg_lambda=1,
                                      scale_pos_weight=1, seed=None,
                                      silent=None, subsample=0.8, verbosity=1),
             iid='warn', n_jobs=None,
             param_grid={'feature_selector': ['cyclic', 'greedy', 'thrifty'],
                         'max_depth': [3, 5, 7],
                  

In [28]:
pd.DataFrame(xgb_rf_reg_gscv.cv_results_)[['mean_test_score']]

,mean_test_score
0,0.633117
1,0.633117
2,0.633117
3,0.633117
4,0.678064
5,0.678064
6,0.678064
7,0.678064
8,0.687123
9,0.687123
